In [ ]:
pip install mistralai

In [ ]:
### CLASSES/ATTRIBUTES ###
import csv
import time
from mistralai import Mistral

# Clé API Mistral
MISTRAL_API_KEY = "Add your key API"  # Remplacez par votre clé API Mistral

# Initialiser le client Mistral
client = Mistral(api_key=MISTRAL_API_KEY)

# Chemin du fichier des descriptions
descriptions_file = "Description of metamodel classes.txt"

# Charger les descriptions (en ignorant l'en-tête)
with open(descriptions_file, newline='') as f:
    descriptions = [row[0] for row in csv.reader(f)][1:]

# Préparer le fichier de sortie
with open('rgpd_evaluation_descriptions.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Description", "Covered", "Reasoning"])

    model = "mistral-large-latest"
    initial_sleep_time = 2
    for description in descriptions:
        prompt = f"""
        Task: GDPR Compliance Evaluation

        Description: {description}

        Analyze whether this description (class description and attributes) aligns with the requirements of any GDPR chapter.

        Output format:
        - Covered: [Yes/No].
        - Reasoning: If covered, mention the relevant GDPR chapter and explain why (cites the articles to which the class description belongs). If not, explain why no direct match exists.

        Instructions:
        - Carefully examine each description of the metamodel before responding.
        - Respect the output format strictly.
        - Focus solely on the provided description and evaluate if it aligns with any GDPR chapter's requirements, obligations, or principles.
        - Take into account semantics, i.e. class and attribute definitions and not just their names.
        - Analyze each item individually and provide a precise answer. When your answer is 'Yes', list all items that align with the description, detailing why they match
        """

        max_retries = 5

        for attempt in range(max_retries):
            try:
                response = client.chat.complete(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0
                )

                response_text = response.choices[0].message.content.strip()

                #Extract the response : 'Covered' et 'Reasoning'
                covered, reasoning = None, None
                for line in response_text.splitlines():
                    if "covered:" in line.lower():
                        covered = line.split(":", 1)[1].strip()
                    elif "reasoning:" in line.lower():
                        reasoning = line.split(":", 1)[1].strip()

                if not covered or not reasoning:
                    writer.writerow([description, "Error", "Incorrect or missing format"])
                    print(f"Format error for '{description}'")
                else:
                    result = "Yes" if "yes" in covered.lower() else "No"
                    writer.writerow([description, result, reasoning])
                break  # Sortir de la boucle si la requête a réussi

            except Exception as e:
                # If you get an error, try again (or save the error if this is your last attempt).
                if attempt == max_retries - 1:
                    writer.writerow([description, "Error", f"Erreur: {str(e)}"])
                    print(f"Erreur pour '{description}': {e}")
                else:
                    sleep_time = initial_sleep_time * (attempt + 1)
                    print(f"Erreur pour '{description}', tentative {attempt + 1}. Nouvelle tentative dans {sleep_time} secondes.")
                    time.sleep(sleep_time)

print("Evaluation completed and results saved in rgpd_evaluation_descriptions.csv")


In [ ]:
### USER STORIES ###
import csv
import time
from mistralai import Mistral

MISTRAL_API_KEY = "Add your key API"

client = Mistral(api_key=MISTRAL_API_KEY)

user_stories_file = "User stories.txt"

with open(user_stories_file, newline='') as f:
    user_stories = [row[0] for row in csv.reader(f)][1:]  # Ignorer l'en-tête

with open('rgpd_evaluation_user stories.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["User Story", "Covered", "Reasoning"])

    model = "mistral-large-latest"

    for user_story in user_stories:
        prompt = f"""
        Task: GDPR Compliance Evaluation

        User Story: {user_story}

        Analyze whether the user story aligns with the requirements of any GDPR chapter.

        Output format:
        - Covered: "Yes" if the user story is covered by a GDPR chapter, otherwise "No".
        - Reasoning: If covered, mention the relevant GDPR chapter and explain why (cites the articles to which the user story belongs). If not, explain why no direct alignment exists.

        Instructions:
        - Respect the output format strictly.
        - Focus solely on the provided description and evaluate if it aligns with any GDPR chapter's requirements, obligations, or principles.
        """

        time.sleep(5)
        response = client.chat.complete(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )

        response_text = response.choices[0].message.content.strip()

        # Extract coverage and reasoning from the answer
        covered, reasoning = None, None
        for line in response_text.splitlines():
            if "covered:" in line.lower():
                covered = line.split(":", 1)[1].strip()
            elif "reasoning:" in line.lower():
                reasoning = line.split(":", 1)[1].strip()

        if not covered or not reasoning:
            writer.writerow([user_story, "Error", "Incorrect or missing format"])
        else:
            writer.writerow([user_story, covered, reasoning])

print("Evaluation completed and results saved in rgpd_evaluation.csv")


In [ ]:
### CONSTRAINTS ###
import csv
import time
from mistralai import Mistral

# Clé API Mistral
MISTRAL_API_KEY = "Add your key API"

client = Mistral(api_key=MISTRAL_API_KEY)

constraints_file = "Constraints.txt"

# Charger les contraintes
with open(constraints_file, newline='') as f:
    constraints = [row[0] for row in csv.reader(f)][1:]  # Ignorer l'en-tête

# Préparer le fichier de sortie
with open('rgpd_evaluation_constraints.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Constraint", "Covered", "Reasoning"])

    model = "mistral-large-latest"

    # Processing each constraint
    for constraint in constraints:
        prompt = f"""
        Task: GDPR Compliance Evaluation

        Constraint: {constraint}

        Analyze whether this constraint aligns with the requirements of any GDPR chapter.

        Output format:
        - Covered: "Yes" if the constraint is covered by a GDPR chapter, otherwise "No".
        - Reasoning: If covered, mention the relevant GDPR chapter and explain why cites the articles to which the constraint belongs. If not, explain why no match exists.

        Instructions:
        - Focus solely on the provided constraint and evaluate if it aligns with any GDPR article's requirements, obligations, or principles.

        """
        max_retries = 5
        for attempt in range(max_retries):
            try:
                # Apply to the Mistral model
                response = client.chat.complete(
                    model=model,
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0
                )

                response_text = response.choices[0].message.content.strip()

                # Extract coverage and reasoning from the answer
                covered, reasoning = None, None
                for line in response_text.splitlines():
                    if "covered:" in line.lower():
                        covered = line.split(":", 1)[1].strip()
                    elif "reasoning:" in line.lower():
                        reasoning = line.split(":", 1)[1].strip()

                # Write to CSV file based on response
                if not covered or not reasoning:
                    writer.writerow([constraint, "Error", "Incorrect or missing format"])
                    print(f"Format error for '{constraint}'")
                else:
                    result = "Yes" if "yes" in covered.lower() else "No"
                    writer.writerow([constraint, result, reasoning])
                break  # Exit the loop if everything works

            except Exception as e:
                if attempt == max_retries - 1:
                    writer.writerow([constraint, "Error", f"Erreur: {str(e)}"])
                    print(f"Erreur pour '{constraint}': {e}")
                else:
                    time.sleep(5 * (attempt + 1))

print("Evaluation completed and results saved in  rgpd_evaluation_constraints.csv")
